In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from mlelec.data.dataset import precomputed_molecules, MoleculeDataset, MLDataset
import torch
from ase.io import read
import ase
from mlelec.models.linear import LinearTargetModel

In [2]:

frames = read('../../examples/data/water_1000/water_1000.xyz', ':4')
for f in frames: 
    f.pbc = False
h = torch.randn((4,10,10)) 
h=h+h.transpose(-1,-2)
over = torch.zeros(4,10,10)
# orbs = {1:[[1,0,0],[2,0,0],[2,1,1],[2,1,-1],[2,1,0]]}
orbs = {1:[[1,0,0]], 8:[[1,0,0], [2,0,0], [2,1,1], [2,1,-1], [2,1,0]]}
# h2_frames= []
# for i, f in enumerate(frames):
#     h2_frames.append(ase.Atoms('H2', positions = f.positions[-2:]))


In [3]:
import hickle
hickle.load('/home/nigam/scratch/MAC/mlelec/examples/data/water_1000/sto-3g/orbitals.hickle')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/home/nigam/scratch/MAC/mlelec/examples/data/water_1000/sto-3g/orbitals.hickle', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [5]:
h_data = MoleculeDataset(mol_name='water_1000', data_path = '../../examples/data/water_1000/sto-3g', aux_path = '../../examples/data/water_1000/sto-3g', device='cuda', aux=['overlap', 'orbitals']) #frames =frames, frame_slice=':4', target_data={'fock': h}, aux=['overlap', 'orbitals'],aux_data = {'overlap': over, 'orbitals':orbs }
h_ml = MLDataset(molecule_data=h_data, device ='cuda')

Loading structures
../../examples/data/water_1000/sto-3g/fock.hickle
['overlap', 'orbitals']
[Errno 2] Unable to synchronously open file (unable to open file: name = '../../examples/data/water_1000/sto-3g/orbitals.hickle', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


KeyError: 8

In [6]:
# h_ml._shuffle(random_seed=5381)
h_ml._split_indices(train_frac=0.7, val_frac=0.2)

NameError: name 'h_ml' is not defined

## training on random data for now 

In [ ]:
linmod = LinearTargetModel(dataset = h_ml, metrics = "l2_loss", nlayers = 1, nout = 1, nhidden = 10, bias = False, device = 'cuda')

In [ ]:
optimizer = torch.optim.Adam(linmod.parameters(), lr=0.01)
for epoch in range(100):
    loss = linmod.forward()
    loss.backward()
    optimizer.step()

    if epoch%10 == 0:
        print(loss)

The data to begin with was random and did not respect the symmetry - so no wonder losses are high

## Predictions

In [ ]:
linmod.forward()
fock = linmod.reconstructed_tensor
print(fock.shape)

## plugging in predicted fock matrix into pyscfad

In [ ]:

import os
os.environ['PYSCFAD_BACKEND']='torch'

import torch
from pyscf import gto

from pyscfad import numpy as np
from pyscfad import ops
from pyscfad.ml.scf import hf
import pyscf.pbc.tools.pyscf_ase as pyscf_ase


In [ ]:

mol = gto.Mole()
mol.atom = pyscf_ase.ase_atoms_to_pyscf(h2_frames[0])
mol.basis = 'def2svp'
mol.build()
fock = linmod.reconstructed_tensor[0]


mf = hf.SCF(mol)

mo_energy, mo_coeff = mf.eig(fock, s = torch.eye(fock.shape[-1], dtype = fock.dtype))
mo_occ = mf.get_occ(mo_energy) # get_occ returns a numpy array
mo_occ = ops.convert_to_tensor(mo_occ)


In [ ]:

dm1 = mf.make_rdm1(mo_coeff, mo_occ)
dip = mf.dip_moment(dm=dm1)
dip_norm = np.linalg.norm(dip)
dip_norm.backward()
print(fock.grad)

In [ ]:
mocc = mo_coeff[:, mo_occ>0]
dm = np.dot(mocc*mo_occ[mo_occ>0], mocc.T)

In [ ]:
print(mocc.dtype)
print(mo_occ[mo_occ>0].dtype)

In [ ]:
print(mocc.conj().T.dtype)